Download GNPS and <i>in silico</i> structural annotation (NAP, Dereplicator, Varquest) data <br>
Replace Job IDs below with your GNPS, Varquest or Dereplicator job IDs:

In [1]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=d23ed2e65ef24eb896b9abebce5243e6&view=download_cytoscape_data' -o GNPS_output_graphML.zip
! unzip -d GNPS_output_graphML/ GNPS_output_graphML.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1140k    0 1140k    0     0   431k      0 --:--:--  0:00:02 --:--:--  431k
Archive:  GNPS_output_graphML.zip
  inflating: GNPS_output_graphML/METABOLOMICS-SNETS-d23ed2e6-download_cytoscape_data-main.graphml  
  inflating: GNPS_output_graphML/params.xml  
  inflating: GNPS_output_graphML/clusterinfo/51383623af2a47f1be504d334f85f7a0.clusterinfo  
  inflating: GNPS_output_graphML/clusterinfosummary/90dff4a84f0843e2943bafdee7c174a1.clustersummary  
  inflating: GNPS_output_graphML/clusterinfogroup/3b2829bcb15d497b93d47df2ad26184a.clusterinfo  
  inflating: GNPS_output_graphML/clusterinfosummarygroup/257a6b709c8f48eeb461dd542288c413.clustersummary  
  inflating: GNPS_output_graphML/clusterinfosummarygroup_attributes/2d61999d163443138ab8814e6d850b25..out  
  inflating: GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs/3

In [2]:
! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=4d971b8162644e869a68faa35f01b915&view=view_significant' -o Varquest_output.zip
! unzip -d Varquest_output/ Varquest_output.zip

! curl -d "" 'https://gnps.ucsd.edu/ProteoSAFe/DownloadResult?task=c62d3283752f4f98b1720d0a6d1ee65b&view=view_significant' -o Derep_output.zip
! unzip -d Derep_output/ Derep_output.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30644    0 30644    0     0  25138      0 --:--:--  0:00:01 --:--:-- 25138
Archive:  Varquest_output.zip
  inflating: Varquest_output/DEREPLICATOR-4d971b81-view_significant-main.tsv  
  inflating: Varquest_output/params.xml  
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1348  100  1348    0     0   1606      0 --:--:-- --:--:-- --:--:--  1604
Archive:  Derep_output.zip
  inflating: Derep_output/DEREPLICATOR-c62d3283-view_significant-main.tsv  
  inflating: Derep_output/params.xml  


load libraries

In [3]:
from pyMolNetEnhancer import *
import pandas as pd
import os
import csv 
import json
from networkx import *
path = '/Applications/MarvinSuite/bin/' #path to MarvinSuite's molconvert used to convert SMILES to InchiKeys
os.environ['PATH'] += ':'+path

In [4]:
netfile = 'GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/' + str(os.listdir('GNPS_output_graphML/clusterinfosummarygroup_attributes_withIDs_withcomponentID/')[0])
gnpslibfile = 'GNPS_output_graphML/result_specnets_DB/'+ str(os.listdir('GNPS_output_graphML/result_specnets_DB/')[0])

nap_id = 'cb63770fe307410492468f62f9edb8f3'

nap = pd.read_csv("http://proteomics2.ucsd.edu/ProteoSAFe/DownloadResultFile?task="+nap_id+"&block=main&file=final_out/node_attributes_table.tsv", sep = "\t")
gnpslib = pd.read_csv(gnpslibfile, sep='\t')
derep = pd.read_csv('Derep_output/'+[s for s in os.listdir('Derep_output') if "DEREPLICATOR" in s][0], sep = '\t')
varquest = pd.read_csv('Varquest_output/'+[s for s in os.listdir('Varquest_output') if "DEREPLICATOR" in s][0], sep = '\t')
matches = [gnpslib, nap, derep, varquest]

In [5]:
out = unique_smiles(matches)

In [6]:
out['df'].to_csv("SMILES.csv", quoting=csv.QUOTE_NONE, escapechar='&')

convert SMILES to InchiKeys

In [7]:
! molconvert inchikey:SAbs SMILES.csv{csv:strucSMILES} -o InchiKeys.txt

In [8]:
ikeys = pd.read_csv("InchiKeys.txt",  sep='\t',header = None)

out['df']["inchikey"] = ikeys
inchi_dic = make_inchidic(out)

ikeys.columns = ["InChIKey"]
ikeys.to_csv("InchiKeys.txt", quoting=csv.QUOTE_NONE, escapechar='&')

retrieve ClassyFire classifcations

In [9]:
get_classifications("InchiKeys.txt")

1000
2000


In [10]:
with open("all_json.json") as tweetfile:
    jsondic = json.loads(tweetfile.read())


df = make_classy_table(jsondic)
df = df.rename(columns = {'class':'CF_class','smiles':'SMILES'})
net = pd.read_csv(netfile,  sep='\t')


final = molfam_classes(net,df,inchi_dic)

In [11]:
final.head()

,cluster index,CF_componentindex,CF_NrNodes,CF_kingdom,CF_kingdom_score,CF_superclass,CF_superclass_score,CF_class,CF_class_score,CF_subclass,CF_subclass_score,CF_Dparent,CF_Dparent_score,CF_MFramework,CF_MFramework_score
0,4025,S1,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
1,4022,S2,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
2,4020,S3,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
3,5989,S4,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,
4,5983,S5,1,no matches,,no matches,,no matches,,no matches,,no matches,,no matches,


write ClassyFire results per molecular family to file. To visualize chemical classes within the mass spectral molecular network, this file can be imported as table into Cytoscape.

In [12]:
final.to_csv("ClassyFireResults_Network.txt", sep = '\t', index = False)

create graphML file

In [13]:
graphMLfile = 'GNPS_output_graphML/' + [x for x in os.listdir('GNPS_output_graphML/') if 'METABOLOMICS' in x][0]

In [14]:
graphML = read_graphml(graphMLfile)

In [15]:
graphML_classy = make_classyfire_graphml(graphML,final)

write graphML file

In [17]:
nx.write_graphml(graphML_classy, "ClassyFireResults_Network.graphml", infer_numeric_types = True)